<h1> Interpret question and return relevent results from reviews</h1>

In [416]:
import gensim
#from sklearn.datasets import fetch_20newsgroups
import logging
from collections import defaultdict
from gensim import corpora, models, similarities
#import stopwords
#from pattern.en import lemma
import re
import psycopg2

In [417]:
conn = psycopg2.connect("host=localhost port=5432 dbname=amazon user=postgres password=darkmatter")
cur = conn.cursor()

<h1> clean up each question/answer and tokenize </h1>

In [6]:
def process_line(sentence):
    #step 1 split if we need to
    sentences=re.split(r'[;:.!?]\s*', sentence)
    result= [re.findall("[a-z-.'0-9]+", sent.lower()) for sent in sentences if \
            re.findall("[a-z-.'0-9]+", sent.lower())!=[]]
    if result==[]:
        result=[['']]
    return result
stoplist = set('for a of the and to in rt'.split())

<h1> define stop words to remove from the question model and apply </h1>

In [7]:
stoplist = set('a an for of the and to in rt'.split())
stoplist

{'a', 'an', 'and', 'for', 'in', 'of', 'rt', 'the', 'to'}

<h3> Use word2vec trained on the question model to remove the irrelavent words from the topic </h3>

In [251]:
QmodelB=models.Word2Vec.load('/home/ubuntu/TallLabs/models/QmodelB')
RmodelB=models.Word2Vec.load('/home/ubuntu/TallLabs/models/RmodelB')
bag_of_words='is,will,wil,may,might,does,do,can,could,must,should,are,would,did,need,take,out,how,would,am,at,\
anyone,has,have,off,that,which,who,please,thank,you,that,fit,these,they,many,work,with,time,turn,fit,fitt,\
from,hard,use,your,not,into,non,hold,say,from,one,two,like,than,same,thanks,find,make,hot,be,as,well,there,\
son,daughter,amazon,when,after,change,both,ask,know,help,me,recently,purchased,item,any,newest,or'.split(',')
bag_of_words_verbs='is,will,wil,may,might,does,do,can,could,must,should,are,would,did,take,out,would,\
anyone,off,that,which,who,please,thank,you,that,these,they,many,time,turn,newest,there,am,at,\
from,hard,use,your,not,into,non,hold,say,from,one,two,like,than,same,thanks,\
son,daughter,amazon,when,after,change,both,ask,know,help,me,recently,purchased,item,any'.split(',')
complete_bag=set(sum([[item[0] for item in QmodelB.most_similar(word)] for word in bag_of_words],[]))|stoplist|set(bag_of_words)
complete_bag_verbs=set(sum([[item[0] for item in QmodelB.most_similar(word)] for word in bag_of_words_verbs],[]))|stoplist|set(bag_of_words_verbs)

#complete_bag

def q_filter(sentence):
    return [word.lower() for word in sum(process_line(sentence),[]) if word not in complete_bag]

def q_filter_verb(sentence):
    return [word.lower() for word in sum(process_line(sentence),[]) if word not in complete_bag_verbs]

def find_bigrams(key_words):
    ii=0
    while ii < len(key_words)-1:
        if key_words[ii]+'_'+key_words[ii+1] in QmodelB:
            key_words.insert(ii,key_words[ii]+' '+key_words[ii+1])
            key_words.pop(ii+1)
            key_words.pop(ii+1)
        ii+=1
    return key_words
def return_key_words(question):
    question=question.lower()
    key_words_action= find_bigrams(q_filter_verb(question))
    key_words=find_bigrams(q_filter(question))
    [key_words_action.remove(word) for word in key_words]
    return key_words, key_words_action

def find_relevent_sentence(text,key_words):
    text=text.lower()
    text=text.replace('/',' / ').replace('(',' ( ').replace(')',' ) ')
    good_sen=[]
    good_qual=[]
    good_qual_val=[]
    sentences = re.split(r"(?<![0-9])[.?!;](?![0-9])",text) #whatever delimiters you will need
    for sen in sentences:
        if(set(key_words) & set(sen.split())): #find the intersection/union
            good_sen.append(sen)
            good_qual.append(str(len(set(key_words) & set(sen.split())))+'/'+str(len(set(key_words))))
            good_qual_val.append(len(set(key_words) & set(sen.split()))/len(set(key_words)))
    return good_sen,good_qual,good_qual_val

def merge_review(sql_result):
    reviews=[]
    [reviews.append(review[0]) for review in sql_result]
    return' '.join(reviews)

In [38]:
def check_key(word,mod):
    try:
        return mod.most_similar(word,topn=5)
    except KeyError:
        return [['']]

<h1> given a sentence isolate the key words about the topic </h1>

In [13]:
question='can you play these on a iphone c and do i need a charger on a world map?'
question='I would like to know. does it work on Audi A4 2.0? thank you!'
question='Can you please help me. I recently purchased this item on amazon. Dos the lense work with a nikon? Thanks'
question='Does anyone know. I recently purchased this item on amazon. Dos the lense work with a nikon? Thanks'
key_words, key_words_action = return_key_words(question)

In [14]:
key_words, key_words_action

(['lense', 'nikon'], ['work', 'with'])

<h3> select a question </h3>

In [9]:
cur.execute("SELECT id,asin,question from qa where id=%s limit 1;",(90000,))
result=cur.fetchall()[0]
qa_id=result[0]
asin=result[1]
question=result[2]
question

'Do these fit the 2015 camry?'

<h1> Select a question with a known asin </h1>

In [431]:
asin='8288878881'
cur.execute("SELECT question from qa where asin=%s limit 1;",(asin,))
result=cur.fetchall()[0]
qa_id=result[0]
asin=result[1]
question=result[2]
question

'How long is the cord?'

In [419]:
key_words, key_words_action = return_key_words(question)
key_words, key_words_action

(['long', 'cord'], ['how'])

In [420]:
similar_keys=sum([[' '.join(item[0].split('_')) for item in check_key(word,RmodelB) if item!=[''] and item[1]>0.7] \
                  for word in key_words],[])
similar_keys

['power cord', 'chord', 'cable', 'usb cable']

In [384]:
cur.execute("select metajson->'imUrl', metajson->'description', title from metadata where asin=%s and id >1000000 limit 1;",(asin,))
result=cur.fetchall()[0]
image=result[0]
description=result[1]
title=result[2]

In [359]:
find_relevent_sentence(description,key_words)

([], [], [])

In [433]:
cur.execute("select reviewtext from reviews_cell_phones_and_accessories where asin=%s;",(asin,))
result=cur.fetchall()

In [427]:
good_sen,good_qual,good_qual_val=find_relevent_sentence(merge_review(result),key_words)

In [428]:
sorted_index=sorted(range(len(good_qual_val)),key=lambda x:good_qual_val[x])[::-1]

In [429]:
sorted_index

[17, 12, 11, 0, 16, 15, 14, 13, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [430]:
'\n'.join([good_qual[index]+':'+good_sen[index] for index in sorted_index][0:5])

'2/2:  it charges my phone quickly and the cord is super long\n2/2: now i have a long cord dangling everywhere\n2/2: instead, i got a 5 foot cord which is a bit long for the purpose i was going to use it for  ( in my car ) \n2/2:works great nice long cord\n1/2: maybe, a 2 21 / 2 foot cord'

<h3> print all of the reviews </h3>

In [439]:
print('\n\n'.join([val[0] for val in result]))

Works great nice long cord. Does the trick. Bought several of these to put in diff. rooms & one for work.

Good charger and is perfect because it is also has a USB connection. Also love that it is original material it works like that too giving a quick charge when i need it.

I have had 3 phone chargers go bad in two weeks. I can not say where I purchased them. Most chargers for cell phones are cheaply made. This one is OK I guess.

Got this to replace a frayed OEM cord. The cord itself is ok ;however, the plug that looks like OEM and is stamped as such, is crap! Took 8 times longer to charge with this unit! When the cord was paired with the actual OEM plug in, it worked fine. So if you need a cord its ok; however, it is definitely not OEM and the plug is worthless. For the price and really just needing a cord it is a good value but I am very disappointed with the false markings as OEM when it is certainly not.

I received this product in a very timely manner which was delightful, howe

In [437]:
result[0]

('Works great nice long cord. Does the trick. Bought several of these to put in diff. rooms & one for work.',)